In [1]:
import os
import cv2
import torch
import logging
import pandas as pd
import psycopg2


In [2]:
# Logging configuration
logging.basicConfig(
    filename='object_detection.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [3]:
# Paths
IMAGE_DIR = "../data//scraped_images"  # Directory containing images
DETECTION_RESULTS = "../data/detection_results.csv"  # Output CSV file

# Database configuration
DB_CONFIG = {
    'dbname': 'telegram_data',
    'user': 'postgres',
    'password': '12345',  
    'host': 'localhost',
    'port': 5432
}

In [4]:
#Load YOLOv5 model
def load_yolo_model():
    """Load the pre-trained YOLOv5 model."""
    try:
        model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
        logging.info("YOLO model loaded successfully.")
        print("YOLO model loaded successfully.")
        return model
    except Exception as e:
        logging.error(f"Error loading YOLO model: {e}")
        raise e

In [5]:

# Perform Object Detection
def detect_objects(model, image_dir, output_csv):
    """Detect objects in images and save results to a CSV file."""
    try:
        # Get list of image files
        images = [os.path.join(image_dir, img) for img in os.listdir(image_dir) if img.endswith(('.jpg', '.png'))]
        if not images:
            print("No images found for detection.")
            return

        all_results = []
        for image_path in images:
            # Perform detection
            results = model(image_path)
            detections = results.pandas().xyxy[0]  # Get detection data as DataFrame
            
            if not detections.empty:
                detections['image_path'] = image_path
                all_results.append(detections)

        # Save results to CSV
        if all_results:
            results_df = pd.concat(all_results, ignore_index=True)
            results_df.to_csv(output_csv, index=False)
            logging.info(f"Detection results saved to {output_csv}")
            print(f"Detection results saved to {output_csv}")
        else:
            print("No detections found in any image.")
    except Exception as e:
        logging.error(f"Error during detection: {e}")
        print(f"Error during detection: {e}")

In [6]:


# Store Detection Results in Database
def store_results_to_db(csv_file):
    """Store detection results from CSV into PostgreSQL database."""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()

        # Create detection_results table if not exists
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS detection_results (
            id SERIAL PRIMARY KEY,
            image_path TEXT,
            x_min FLOAT,
            y_min FLOAT,
            x_max FLOAT,
            y_max FLOAT,
            confidence FLOAT,
            class_id INTEGER,
            name TEXT
        )
        """)
        conn.commit()

        # Load data from CSV
        results_df = pd.read_csv(csv_file)
        for _, row in results_df.iterrows():
            cursor.execute("""
                INSERT INTO detection_results (image_path, x_min, y_min, x_max, y_max, confidence, class_id, name)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                row['image_path'], row['xmin'], row['ymin'], row['xmax'], row['ymax'],
                row['confidence'], row['class'], row['name']
            ))

        conn.commit()
        conn.close()
        logging.info("Detection results stored in database.")
        print("Detection results stored in database.")
    except Exception as e:
        logging.error(f"Error storing results in database: {e}")
        print(f"Error storing results in database: {e}")


In [7]:

# Main function
if __name__ == "__main__":
    # Load YOLO model
    # model = load_yolo_model()

    # # Perform object detection and save results to CSV
    # detect_objects(model, IMAGE_DIR, DETECTION_RESULTS)

    # Store detection results in database
    store_results_to_db(DETECTION_RESULTS)


Detection results stored in database.
